## Análise de Postagens do Instagram: EDA + Modelagem de Regressão

## Instagram Post Analysis & Predictive Modeling
Etapa 1: Carregar e limpar o conjunto de dados <br>
Remover espaços em branco extras dos nomes das colunas e tipos de postagem
Excluir postagens duplicadas <br>
Criar uma nova pontuação de engajamento = curtidas + 2 × comentários

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv(r'C:\Users\stenz\Documents\Ciência de Dados\Santander BootCamp 2025\01 - Estrutura de dados\07 - ETL\posts.csv')
df.head()

In [ ]:
df['engagements']= df['likes'] + 2 * df['comments']
df.head()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

## Step 2: Exploratory Data Analysis
Distribution of post types <br>
Histograms of likes, comments, and engagement <br>
Boxplots to see spread and outliers by post type <br>
Pairwise correlation between metrics

In [ ]:
sns.countplot(x='Post_Type', data=df, palette='Set2')
plt.title('Distribution of Post Types')
plt.tight_layout()
plt.show()

In [ ]:
for col in ['likes','comments','engagements']:
    sns.histplot(df[col], kde=True, color='skyblue')
    plt.title(f'Distribution of {col}')
    plt.tight_layout()
    plt.show()

In [ ]:
for col in ['likes','comments','engagements']:
    sns.boxplot(x='Post_Type', y=col, data=df)
    plt.title(f'{col} by Post Type')
    plt.tight_layout()
    plt.show()

## Step 3: Total Performance by Post Type
We sum up total likes, comments, and engagement for each post type to evaluate their overall contribution.<br>
Then we re-organize the data to examine each metric (likes/comments/engagement) across all 3 post types — for cleaner comparison.

In [ ]:
df_sum = df.groupby("Post_Type")[['likes', 'comments', 'engagements']].sum().sort_values("engagements", ascending=False)
print("\nTotal Likes, Comments, Engagement by Post Type:\n", df_sum)

df_sum.plot(kind='bar', figsize=(10, 6), title='Total Likes, Comments, Engagement by Post Type')
plt.ylabel("Total Count")
plt.tight_layout()
plt.show()

In [ ]:
metrics = ['likes', 'comments', 'engagements']

df_grouped = df.groupby("Post_Type")[metrics].sum()

for metric in metrics:
    plt.figure(figsize=(8, 5))
    sns.barplot(x=df_grouped.index, y=df_grouped[metric].values)
    plt.title(f"Total {metric} by Post Type")
    plt.xlabel("Post Type")
    plt.ylabel(f"Total {metric}")
    plt.tight_layout()
    plt.show()

## Etapa 4: Modelagem Preditiva com Regressão Linear
Construímos 3 modelos separados para prever:
Curtidas <br>
Comentários <br>
Engajamento <br>
Todos baseados exclusivamente em Post_Type, usando codificação one-hot. <br>
Avaliamos cada modelo usando: <br>
R² (quão bem o modelo explica os dados) <br>
Erro Quadrático Médio (quão distantes as previsões estão) <br>
Coeficientes do modelo (como cada tipo de postagem se compara à linha de base: Carrossel)

In [ ]:
# Verificar versão do scikit-learn
import sklearn
print("Versão do scikit-learn:", sklearn.__version__)

# Importar o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Exemplo: assumindo que df já existe e contém a coluna 'Post_Type'

# Criar o encoder (versões mais novas usam sparse_output)
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Ajustar e transformar os dados
X = encoder.fit_transform(df[['Post_Type']])

# Obter nomes das features
feature_names = encoder.get_feature_names_out(['Post_Type'])

# Transformar em DataFrame (opcional, mas útil no Jupyter)
X_df = pd.DataFrame(X, columns=feature_names)

X_df.head()

In [ ]:
summary_data = []

for metric in metrics:
    y = df[metric]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    row = {
        'Target': metric.capitalize(),
        'R² Score': round(r2, 3),
        'MSE': round(mse),
        'Image Coef': round(model.coef_[0], 2),
        'Reel Coef': round(model.coef_[1], 2),
        'Intercept': round(model.intercept_, 2)
    }

    summary_data.append(row)


summary_df = pd.DataFrame(summary_data)
print(summary_df)

## Etapa 5: As 5 publicações mais engajadas
Por fim, identificamos as 5 publicações mais engajadoras no geral, usando: <br>
engajamento = curtidas + 2 × comentários<br>
Isso nos ajuda a identificar padrões em conteúdo de alto desempenho.

In [ ]:
top_posts = df.nlargest(5, 'engagements')[['Post_id', 'Post_Type', 'likes', 'comments', 'engagements']]
print("\nTop 5 Posts by Engagement:\n", top_posts)

# Caminho do arquivo CSV
caminho_csv = r"C:\Users\stenz\Documents\Ciência de Dados\Santander BootCamp 2025\top_posts.csv"

# Salvar como CSV
top_posts.to_csv(caminho_csv, index=False, encoding='utf-8-sig')

print(f"\nArquivo CSV salvo em:\n{caminho_csv}")

### Considerações finais: <br>
As publicações em carrossel tendem a ter melhor desempenho em curtidas e engajamento. <br>
Os Reels geram significativamente mais comentários do que outros formatos. <br>
O tipo de publicação, por si só, é um fraco indicador de desempenho (R² < 0), <br> sugerindo que precisamos de mais recursos.